In [13]:
# Chạy cell này đầu tiên
from bs4 import BeautifulSoup
from IPython.display import clear_output
import requests
import json
import os

In [21]:
# Sau đó chạy cell này
def getSongInfo(name, url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        page_content = response.text
        soup = BeautifulSoup(page_content, 'html.parser')
        links = soup.select(".play-this-track-playlinks .play-this-track-playlink")
        result = {}
        for link in links:
            result[link.getText().strip()] = link['href']
        return {name: result}
    except requests.exceptions.RequestException as e:
        print(f"Request error: {e}")
        return {name: []}
    except Exception as e:
        print(f"An error occurred: {e}")
        return {name: []}
def crawlAllSongOfArtist(artist):
    try:
        allSongs[artist] = []
        artistQuery = artist.replace(" ", "+")
        url = f"https://www.last.fm/music/{artistQuery}/+tracks?date_preset=ALL"
        response = requests.get(url)
        response.raise_for_status()
        page_content = response.text
        soup = BeautifulSoup(page_content, 'html.parser')
        last_page = int(soup.find_all(attrs={"class": "pagination-page"})[-1].find('a').get_text().strip())
        for i in range(1, last_page+1):
        # For test using: 
        #for i in range(1, 2):
            res = requests.get(f"{url}&page={i}")
            res.raise_for_status()
            pg = res.text
            soup = BeautifulSoup(pg, 'html.parser')
            songs = soup.select("tr.chartlist-row > .chartlist-name > a")
            for song in songs:
            # For test using: 
            #for song in songs[:5]:
                songName = song.getText()
                songUrl = song['href']
                allSongs[artist].append(getSongInfo(songName, f"https://www.last.fm{songUrl}"))
                print(f"artist: {artist} - page: {i}/{last_page+1} - song: {songName}")
                clear_output(wait=True)
    except requests.exceptions.RequestException as e:
        print(f"Request error: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

In [25]:
'''
Bắt đầu từ đây 
1. Đổi lại firstIndex và lastIndex dựa vào danh sách nghệ sĩ được yêu cầu
2. Chạy cell này. Giữ internet ổn định và cắm máy đấy rồi đi làm việc khác
Lưu ý: * Sau mỗi nghệ sĩ crawl thành công sẽ lưu vào file. 
Do đó có thể chạy tiếp cell này sau khi stop, tiến trình sẽ tiếp tục.
Tuy nhiên nên hạn chế để tránh lỗi

* Nếu thấy log bên dưới error, lập tức bấm Interrupt và kiểm tra lại internet
'''
allArtist = []
targetFileData = {}
allSongs = {}
firstIndex = 0
lastIndex = 5
targetFileName = f'./songData{firstIndex}-{lastIndex}.json'
# File danh sách nghệ sĩ
with open('./artistList.json') as user_file:
    file_contents = user_file.read()
    allArtist = list(json.loads(file_contents))

if os.path.exists(targetFileName):
    with open(targetFileName) as resultFile:
        file_contents = resultFile.read()
        targetFileData = dict(json.loads(file_contents))
else:
    with open(targetFileName, 'w') as resultFile:
        json.dump({}, resultFile)

for i in range(firstIndex, lastIndex+1):
    artist = allArtist[i]
    if artist not in targetFileData:
        crawlAllSongOfArtist(artist)
        with open(targetFileName, 'w') as resultFile:
            json.dump({**allSongs, **targetFileData}, resultFile)

artist: 070 Shake - page: 1/11 - song: Skin and Bones


https://api.socialcounts.org/youtube-video-live-view-count/8xg3vE8Ie_E

In [ ]:
allSongs